In [ ]:
import matplotlib.pyplot as plt
from astropy.cosmology import FlatLambdaCDM
from astropy.units import Quantity
from slsim.lens_pop import LensPop
from slsim.Plots.lens_plots import LensingPlots
import numpy as np
from tqdm import tqdm
import corner
import slsim.Pipelines as pipelines
import slsim.Sources as sources
import slsim.Deflectors as deflectors
from slsim.lens import Lens
from numba import njit, prange
from slsim.image_simulation import lens_image, rgb_image_from_image_list
from slsim.Util.param_util import gaussian_psf
# import standard python modules
import copy
import matplotlib.patches as mpatches
from itertools import combinations

%matplotlib inline

# import lenstronomy and hierArc modules
from lenstronomy.Cosmo.lens_cosmo import LensCosmo
from hierarc.Sampling.ParamManager.cosmo_param import CosmoParam
from hierarc.Sampling.mcmc_sampling import MCMCSampler

from hierarc.Likelihood.LensLikelihood.double_source_plane import (
    beta_double_source_plane,
    # beta2theta_e_ratio,
)


from astropy.io import fits
from astropy.table import Table

/Users/paras/Library/CloudStorage/GoogleDrive-paras.sharma@stonybrook.edu/My Drive/SL Research/codes/repos/self/slsim/slsim/Plots/lens_plots.py:5: UserWarning: If you want to simulate images with Roman filters, please install the galsim module.
Note that this module is not supported on Windows
  from slsim.roman_image_simulation import simulate_roman_image


## Generate population of sources and (potential) deflectors

In [ ]:
# define a cosmology
cosmology = "FwCDM"  # Flat wCDM cosmology
# other options are: "FLCDM FwCDM", "w0waCDM", "oLCDM"
kwargs_cosmo_true = {"h0": 70, "om": 0.3, "w": -1}  # cosmological model of the forecast

# create astropy.cosmology instance of input cosmology
cosmo_param = CosmoParam(cosmology=cosmology)
cosmo_true = cosmo_param.cosmo(kwargs_cosmo_true)

# define a sky area
sky_area = Quantity(value=10, unit="deg2")


# define limits in the intrinsic deflector and source population (in addition to the skypy config
# file)
kwargs_deflector_cut = {"band": "g", "band_max": 28, "z_min": 0.01, "z_max": 2.5}
kwargs_source_cut = {"band": "g", "band_max": 28, "z_min": 0.1, "z_max": 5.0}

# Generate galaxy population using skypy pipeline.
galaxy_simulation_pipeline = pipelines.SkyPyPipeline(
    skypy_config=None, sky_area=sky_area, filters=None, cosmo=cosmo_true
)

# Initiate deflector population class.
lens_galaxies = deflectors.AllLensGalaxies(
    red_galaxy_list=galaxy_simulation_pipeline.red_galaxies,
    blue_galaxy_list=galaxy_simulation_pipeline.blue_galaxies,
    kwargs_cut=kwargs_deflector_cut,
    kwargs_mass2light=None,
    cosmo=cosmo_true,
    sky_area=sky_area,
)

# Initiate source population class.
source_galaxies = sources.Galaxies(
    galaxy_list=galaxy_simulation_pipeline.blue_galaxies,
    kwargs_cut=kwargs_source_cut,
    cosmo=cosmo_true,
    sky_area=sky_area,
    catalog_type="skypy",
)

# make galaxy-galaxy population class using LensPop
gg_lens_pop = LensPop(
    deflector_population=lens_galaxies,
    source_population=source_galaxies,
    cosmo=cosmo_true,
    sky_area=sky_area,
)

# make lensplot class for extracting rgb images
gg_plot = LensingPlots(gg_lens_pop, num_pix=100, coadd_years=10)

print(f"LensPop initialized. Potential deflectors: {gg_lens_pop.deflector_number} ; Potential sources: {gg_lens_pop.source_number}");

/Users/paras/Library/CloudStorage/GoogleDrive-paras.sharma@stonybrook.edu/My Drive/SL Research/codes/repos/self/slsim/slsim/Deflectors/all_lens_galaxies.py:54: UserWarning: Angular size is converted to arcsec because provided input_catalog_type is skypy. If this is not correct, please refer to the documentation of the class you are using
  red_galaxy_list = catalog_with_angular_size_in_arcsec(
/Users/paras/Library/CloudStorage/GoogleDrive-paras.sharma@stonybrook.edu/My Drive/SL Research/codes/repos/self/slsim/slsim/Deflectors/all_lens_galaxies.py:57: UserWarning: Angular size is converted to arcsec because provided input_catalog_type is skypy. If this is not correct, please refer to the documentation of the class you are using
  blue_galaxy_list = catalog_with_angular_size_in_arcsec(


LensPop initialized. Potential deflectors: 3514838 ; Potential sources: 4746379


In [ ]:
# kwargs_lens_cut = {"min_image_separation": 1, "max_image_separation": 10}
kwargs_lens_cut = {}
selected_lenses = gg_lens_pop.draw_population(
    kwargs_lens_cuts=kwargs_lens_cut,
)
print(f"Number of selected lenses: {len(selected_lenses)}")

Number of selected lenses: 3445


In [ ]:
# make pairs of deflector and source galaxies and find the measured ratios of the Einstein radius
lens_pairings = combinations(
    selected_lenses, 2
)  # all possible pairs of lenses
lens_pairings = list(lens_pairings)

pairing_parameters = {
    "lens_pairings": [],
    "z_D1": [],
    "z_D2": [],
    "z_S1": [],
    "z_S2": [],
    "theta_E1": [],
    "theta_E2": [],
    "theta_E_ratio": [],
    "theta_E1_virtual": [], # virtual Einstein radius of lens1 for source2
    "theta_E2_virtual": [], # virtual Einstein radius of lens2 for source1
    "theta_E1_by_theta_E2_virtual": [], # ratio of virtual Einstein radii
    "theta_E1_virtual_by_theta_E2": [], # ratio of virtual Einstein radii
    "sigma_v_D1": [], # velocity dispersion of deflector 1
    "sigma_v_D2": [], # velocity dispersion of deflector 2
    "beta_DSPL_D1": [], # beta double source plane i.e., ratio of distances, assuming D1
    "beta_DSPL_D2": [], # beta double source plane i.e., ratio of distances, assuming D2
    "delta_z_D": [], # difference in redshift of deflector 1 and 2
    "delta_sigma_v_D": [], # difference in velocity dispersions
    "abs_delta_z_D": [], # absolute difference in redshift of deflector 1 and 2
    "abs_delta_sigma_v_D": [], # absolute difference in velocity dispersions
    "mag_S1_i": [],
    "mag_S2_i": [],
    "mag_S1_r": [],
    "mag_S2_r": [],
    "mag_S1_g": [],
    "mag_S2_g": [],
    "mag_S1_z": [],
    "mag_S2_z": [],
    "mag_S1_y": [],
    "mag_S2_y": [],
    "mag_D1_i": [],
    "mag_D2_i": [],
    "mag_D1_r": [],
    "mag_D2_r": [],
    "mag_D1_g": [],
    "mag_D2_g": [],
    "mag_D1_z": [],
    "mag_D2_z": [],
    "mag_D1_y": [],
    "mag_D2_y": [],
    "size_D1": [],
    "size_D2": [],
}

for i in tqdm(range(len(lens_pairings)), desc="Calculating lens pair parameters", total=len(lens_pairings)):
    lens_pair = lens_pairings[i]
    lens1 = lens_pair[0]
    lens2 = lens_pair[1]

    source1 = lens1.source(index = 0)
    source2 = lens2.source(index = 0)

    deflector1 = lens1.deflector
    deflector2 = lens2.deflector

    # get the deflector and source redshifts
    z_D1 = lens1.deflector_redshift
    z_D2 = lens2.deflector_redshift
    z_S1 = source1.redshift
    z_S2 = source2.redshift

    # get the Einstein radii
    theta_E1 = lens1._einstein_radius(source1)
    theta_E2 = lens2._einstein_radius(source2)

    # get the velocity dispersions
    sigma_v_D1 = deflector1.velocity_dispersion()
    sigma_v_D2 = deflector2.velocity_dispersion()

    # calculate the ratio of the Einstein radii
    theta_E_ratio = theta_E1 / theta_E2

    # calculate the virtual Einstein radii
    theta_E1_virtual = lens2._einstein_radius(source1) # virtual Einstein radius of lens2 for source1
    theta_E2_virtual = lens1._einstein_radius(source2) # virtual Einstein radius of lens1 for source2

    # calculate the ratio of the virtual Einstein radii
    theta_E1_by_theta_E2_virtual = theta_E1 / theta_E2_virtual
    theta_E1_virtual_by_theta_E2 = theta_E1_virtual / theta_E2

    # calculate the beta double source plane
    if z_D1 < np.min([z_S1, z_S2]) and z_D2 < np.min([z_S1, z_S2]):
        beta_DSPL_D1 = beta_double_source_plane(
            z_D1, z_S1, z_S2, cosmo_true
        )
        beta_DSPL_D2 = beta_double_source_plane(
            z_D2, z_S1, z_S2, cosmo_true
        )
    elif z_D1 < np.min([z_S1, z_S2]) and z_D2 > np.max([z_S1, z_S2]):
        beta_DSPL_D1 = beta_double_source_plane(
            z_D1, z_S1, z_S2, cosmo_true
        )
        beta_DSPL_D2 = np.nan
    elif z_D1 > np.max([z_S1, z_S2]) and z_D2 < np.min([z_S1, z_S2]):
        beta_DSPL_D1 = np.nan
        beta_DSPL_D2 = beta_double_source_plane(
            z_D2, z_S1, z_S2, cosmo_true
        )
    else:
        beta_DSPL_D1 = np.nan
        beta_DSPL_D2 = np.nan

    # calculate the difference in redshift and velocity dispersion
    delta_z_D = z_D1 - z_D2
    delta_sigma_v_D = sigma_v_D1 - sigma_v_D2
    abs_delta_z_D = np.abs(delta_z_D)
    abs_delta_sigma_v_D = np.abs(delta_sigma_v_D)

    # append the values to the pairing_parameters dictionary
    pairing_parameters["lens_pairings"].append(lens_pair)
    pairing_parameters["z_D1"].append(z_D1)
    pairing_parameters["z_D2"].append(z_D2)
    pairing_parameters["z_S1"].append(z_S1)
    pairing_parameters["z_S2"].append(z_S2)
    pairing_parameters["theta_E1"].append(theta_E1)
    pairing_parameters["theta_E2"].append(theta_E2)
    pairing_parameters["theta_E_ratio"].append(theta_E_ratio)
    pairing_parameters["theta_E1_virtual"].append(theta_E1_virtual)
    pairing_parameters["theta_E2_virtual"].append(theta_E2_virtual)
    pairing_parameters["theta_E1_by_theta_E2_virtual"].append(
        theta_E1_by_theta_E2_virtual
    )
    pairing_parameters["theta_E1_virtual_by_theta_E2"].append(
        theta_E1_virtual_by_theta_E2
    )
    pairing_parameters["sigma_v_D1"].append(sigma_v_D1)
    pairing_parameters["sigma_v_D2"].append(sigma_v_D2)
    pairing_parameters["beta_DSPL_D1"].append(beta_DSPL_D1)
    pairing_parameters["beta_DSPL_D2"].append(beta_DSPL_D2)
    pairing_parameters["delta_z_D"].append(delta_z_D)
    pairing_parameters["delta_sigma_v_D"].append(delta_sigma_v_D)
    pairing_parameters["abs_delta_z_D"].append(abs_delta_z_D)
    pairing_parameters["abs_delta_sigma_v_D"].append(abs_delta_sigma_v_D)


    # get the source magnitudes
    mag_S1_i = source1.extended_source_magnitude("i")
    mag_S2_i = source2.extended_source_magnitude("i")
    mag_S1_r = source1.extended_source_magnitude("r")
    mag_S2_r = source2.extended_source_magnitude("r")
    mag_S1_g = source1.extended_source_magnitude("g")
    mag_S2_g = source2.extended_source_magnitude("g")
    mag_S1_z = source1.extended_source_magnitude("z")
    mag_S2_z = source2.extended_source_magnitude("z")
    mag_S1_y = source1.extended_source_magnitude("y")
    mag_S2_y = source2.extended_source_magnitude("y")

    # get the deflector magnitudes
    mag_D1_i = deflector1.magnitude("i")
    mag_D2_i = deflector2.magnitude("i")
    mag_D1_r = deflector1.magnitude("r")
    mag_D2_r = deflector2.magnitude("r")
    mag_D1_g = deflector1.magnitude("g")
    mag_D2_g = deflector2.magnitude("g")
    mag_D1_z = deflector1.magnitude("z")
    mag_D2_z = deflector2.magnitude("z")
    mag_D1_y = deflector1.magnitude("y")
    mag_D2_y = deflector2.magnitude("y")

    # get the deflector sizes
    size_D1 = deflector1.angular_size_light
    size_D2 = deflector2.angular_size_light


    # update pairing_parameters dictionary
    pairing_parameters["mag_S1_i"][i] = mag_S1_i
    pairing_parameters["mag_S2_i"][i] = mag_S2_i
    pairing_parameters["mag_S1_r"][i] = mag_S1_r
    pairing_parameters["mag_S2_r"][i] = mag_S2_r
    pairing_parameters["mag_S1_g"][i] = mag_S1_g
    pairing_parameters["mag_S2_g"][i] = mag_S2_g
    pairing_parameters["mag_S1_z"][i] = mag_S1_z
    pairing_parameters["mag_S2_z"][i] = mag_S2_z
    pairing_parameters["mag_S1_y"][i] = mag_S1_y
    pairing_parameters["mag_S2_y"][i] = mag_S2_y
    pairing_parameters["mag_D1_i"][i] = mag_D1_i
    pairing_parameters["mag_D2_i"][i] = mag_D2_i
    pairing_parameters["mag_D1_r"][i] = mag_D1_r
    pairing_parameters["mag_D2_r"][i] = mag_D2_r
    pairing_parameters["mag_D1_g"][i] = mag_D1_g
    pairing_parameters["mag_D2_g"][i] = mag_D2_g
    pairing_parameters["mag_D1_z"][i] = mag_D1_z
    pairing_parameters["mag_D2_z"][i] = mag_D2_z
    pairing_parameters["mag_D1_y"][i] = mag_D1_y
    pairing_parameters["mag_D2_y"][i] = mag_D2_y
    pairing_parameters["size_D1"][i] = size_D1
    pairing_parameters["size_D2"][i] = size_D2



# convert the pairing_parameters dictionary to a numpy array
pairing_parameters = {
    key: np.array(value) for key, value in pairing_parameters.items()
}

Calculating lens pair parameters:   0%|          | 0/5932290 [00:00<?, ?it/s]/var/folders/4r/jq9w4fy92y9_bqzvdfk2d5h40000gn/T/ipykernel_44536/4269706569.py:63: RuntimeWarning: divide by zero encountered in scalar divide
  theta_E1_by_theta_E2_virtual = theta_E1 / theta_E2_virtual
Calculating lens pair parameters: 100%|██████████| 5932290/5932290 [1:22:57<00:00, 1191.88it/s]


In [ ]:
# save the pairing_parameters dictionary to a fits file
copy_pairing_parameters_ = copy.deepcopy(pairing_parameters)
copy_pairing_parameters_.pop("lens_pairings");

pairing_parameters_table = Table(copy_pairing_parameters_)

# define units for the table
pairing_parameters_table["z_D1"].unit = "dimensionless"
pairing_parameters_table["z_D2"].unit = "dimensionless"
pairing_parameters_table["z_S1"].unit = "dimensionless"
pairing_parameters_table["z_S2"].unit = "dimensionless"
pairing_parameters_table["theta_E1"].unit = "arcsec"
pairing_parameters_table["theta_E2"].unit = "arcsec"
pairing_parameters_table["theta_E_ratio"].unit = "dimensionless"
pairing_parameters_table["theta_E1_virtual"].unit = "arcsec"
pairing_parameters_table["theta_E2_virtual"].unit = "arcsec"
pairing_parameters_table["theta_E1_by_theta_E2_virtual"].unit = "dimensionless"
pairing_parameters_table["theta_E1_virtual_by_theta_E2"].unit = "dimensionless"
pairing_parameters_table["sigma_v_D1"].unit = "km/s"
pairing_parameters_table["sigma_v_D2"].unit = "km/s"
pairing_parameters_table["beta_DSPL_D1"].unit = "dimensionless"
pairing_parameters_table["beta_DSPL_D2"].unit = "dimensionless"
pairing_parameters_table["delta_z_D"].unit = "dimensionless"
pairing_parameters_table["delta_sigma_v_D"].unit = "km/s"
pairing_parameters_table["abs_delta_z_D"].unit = "dimensionless"
pairing_parameters_table["abs_delta_sigma_v_D"].unit = "km/s"
pairing_parameters_table["mag_S1_i"].unit = "mag"
pairing_parameters_table["mag_S2_i"].unit = "mag"
pairing_parameters_table["mag_S1_r"].unit = "mag"
pairing_parameters_table["mag_S2_r"].unit = "mag"
pairing_parameters_table["mag_S1_g"].unit = "mag"
pairing_parameters_table["mag_S2_g"].unit = "mag"
pairing_parameters_table["mag_S1_z"].unit = "mag"
pairing_parameters_table["mag_S2_z"].unit = "mag"
pairing_parameters_table["mag_S1_y"].unit = "mag"
pairing_parameters_table["mag_S2_y"].unit = "mag"
pairing_parameters_table["mag_D1_i"].unit = "mag"
pairing_parameters_table["mag_D2_i"].unit = "mag"
pairing_parameters_table["mag_D1_r"].unit = "mag"
pairing_parameters_table["mag_D2_r"].unit = "mag"
pairing_parameters_table["mag_D1_g"].unit = "mag"
pairing_parameters_table["mag_D2_g"].unit = "mag"
pairing_parameters_table["mag_D1_z"].unit = "mag"
pairing_parameters_table["mag_D2_z"].unit = "mag"
pairing_parameters_table["mag_D1_y"].unit = "mag"
pairing_parameters_table["mag_D2_y"].unit = "mag"
pairing_parameters_table["size_D1"].unit = "radian"
pairing_parameters_table["size_D2"].unit = "radian"

In [ ]:
# save the table to a fits file


table_hdu = fits.BinTableHDU(pairing_parameters_table)
table_hdu.header["N_LENSES"] = len(selected_lenses)
table_hdu.header["N_PAIRS"] = len(lens_pairings)
table_hdu.header["COSMO"] = cosmology
table_hdu.header["H0"] = kwargs_cosmo_true["h0"]
table_hdu.header["OM"] = kwargs_cosmo_true["om"]
table_hdu.header["W"] = kwargs_cosmo_true["w"]
# table_hdu.header["SKY_AREA"] = sky_area.value
# table_hdu.header["SKY_AREA_UNIT"] = sky_area.unit
# table_hdu.header["LENS_CUT"] = kwargs_lens_cut
# table_hdu.header["SOURCE_CUT"] = kwargs_source_cut

table_hdu = fits.BinTableHDU(pairing_parameters_table)

table_hdu.writeto(
    "pairing_parameters_hdu_6Mil_pairs.fits",
    overwrite=True,
)

In [ ]:
num_lenses = 3445
num_pairs = 5932290
# define a cosmology
cosmology = "FwCDM"  # Flat wCDM cosmology
# other options are: "FLCDM FwCDM", "w0waCDM", "oLCDM"
kwargs_cosmo_true = {"h0": 70, "om": 0.3, "w": -1}  # cosmological model of the forecast